Лабораторна робота 2
Підготовка та аналіз даних
Наука про дані: підготовчий етап

In [13]:
#importing libs
import urllib.request
import pandas as pd
import requests
import os
import glob
from datetime import datetime

In [39]:
directory = 'csv_lab2'
files = os.listdir(directory)

def download_file(ids):
    date_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    vhi_url = urllib.request.urlopen(url)
    file_name = f'{directory}/NOAA_ID_{ids}_{date_now}.csv'
    out = open(file_name, 'wb')
    out.write(vhi_url.read())
    out.close()
    print(f"Data from id {ids} was downloaded")

def file_exists(directory, id):
    files = os.listdir(directory)
    for file in files:
        if file.startswith(f"NOAA_ID_{id}_") and file.endswith(".csv"):
            return True
    return False
    
if(not os.path.exists(directory)):
        print("Create dir...")  
        os.makedirs(directory)
else:
        print("Dir already exists.")  
    
for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        # files_to_delete = glob.glob(f"{directory}/NOAA_ID_{ids}_*.csv")
        # if files_to_delete is not None:
        #     for file_to_delete in files_to_delete:
        #             os.remove(file_to_delete)
        if file_exists(directory, ids) == False:
            print(f"No data for {ids}")
            download_file(ids)
    else:
        print(f"Download complete")
        break
print(f"Download complete")

Dir already exists.
No data for 10
Data from id 10 was downloaded
Download complete


In [40]:
files = os.listdir(directory)
df_all = pd.DataFrame()

for file in files:
    headers = ['Year', 'Week', ' SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{directory}/{file}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    if file.split('_')[2].isdigit():
        df['area'] = int(file.split('_')[2])
        df = df.drop(columns=['empty'])
        df_all = df_all.dropna(axis=1, how='all')
        df_all = pd.concat([df_all, df], ignore_index=True)
    else:
        None

print(df_all, "\n")

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...     ...   ...    ...     ...    ...    ...    ...   ...
58045  2024  13.0  0.222  291.75  68.15   3.55  35.85     9
58046  2024  14.0  0.239  293.41  68.55   5.59  37.07     9
58047  2024  15.0  0.254  294.70  67.75   9.07  38.40     9
58048  2024  16.0  0.267  295.58  64.81  16.81  40.82     9
58049  2024  17.0  0.277  296.48  60.06  29.15  44.65     9

[58050 rows x 8 columns] 



In [41]:
#change indexes
indexes = ["22", "24", "23", "25", "3", "4", "8", "19", "20", "21", "9", "26", "10", "11",
                            "12", "13", "14", "15", "16", "27", "17", "18", "6", "1", "2", "7", "5"] 
dict = {i+1: indexes[i] for i in range(len(indexes))} 
df_all.area.replace(dict, inplace=True)  
df_all.to_csv(f"{directory}/NOAA_ALL_CSV.csv")  
print(df_all)
df_all['Year'] = df_all["Year"].astype(int)
df_all['area'] = df_all["area"].astype(int)

C:\Users\ethan\AppData\Local\Temp\ipykernel_4580\23149864.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all.area.replace(dict, inplace=True)


       Year  Week    SMN     SMT    VCI    TCI    VHI area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04   21
1      1982   3.0  0.063  263.45  57.30  32.69  44.99   21
2      1982   4.0  0.061  265.10  53.96  28.62  41.29   21
3      1982   5.0  0.058  266.42  46.87  28.57  37.72   21
4      1982   6.0  0.056  267.47  39.55  30.27  34.91   21
...     ...   ...    ...     ...    ...    ...    ...  ...
58045  2024  13.0  0.222  291.75  68.15   3.55  35.85   20
58046  2024  14.0  0.239  293.41  68.55   5.59  37.07   20
58047  2024  15.0  0.254  294.70  67.75   9.07  38.40   20
58048  2024  16.0  0.267  295.58  64.81  16.81  40.82   20
58049  2024  17.0  0.277  296.48  60.06  29.15  44.65   20

[58050 rows x 8 columns]


Ряд VHI для області за вказаний рік;

Пошук екстремумів (min та max) для вказаних областей та
років

In [42]:
# VHI for year and min|max
def find_vhi(area, year):
    df_search = df_all[(df_all.area == area) & (df_all.Year == year)][['Week', 'VHI']]
    print(f"Here are VHI for province {area} in {year}")  
    print(df_search)  
    print("...")

def find_extremums(area, year):
    max_v = df_all[(df_all.Year.astype(int) == int(year)) & (df_all.area == area)]['VHI'].max()  
    min_v = df_all[(df_all.Year.astype(int) == int(year)) & (df_all.area == area)]['VHI'].min()  
    print(f'The MAX value is: {max_v}')  
    print(f'The MIN value is: {min_v}')  
    print("...")
    return


In [43]:
find_vhi(6, 1999)

Here are VHI for province 6 in 1999
       Week    VHI
30953   1.0  35.16
30954   2.0  38.50
30955   3.0  43.96
30956   4.0  48.92
30957   5.0  51.29
30958   6.0  53.40
30959   7.0  54.47
30960   8.0  54.82
30961   9.0  54.11
30962  10.0  51.83
30963  11.0  50.92
30964  12.0  49.08
30965  13.0  47.36
30966  14.0  46.04
30967  15.0  45.47
30968  16.0  48.51
30969  17.0  53.94
30970  18.0  59.28
30971  19.0  59.55
30972  20.0  57.94
30973  21.0  56.88
30974  22.0  56.72
30975  23.0  54.40
30976  24.0  51.42
30977  25.0  49.65
30978  26.0  48.88
30979  27.0  50.75
30980  28.0  52.65
30981  29.0  55.31
30982  30.0  58.89
30983  31.0  62.14
30984  32.0  64.87
30985  33.0  65.57
30986  34.0  67.76
30987  35.0  68.54
30988  36.0  63.39
30989  37.0  57.69
30990  38.0  51.90
30991  39.0  47.76
30992  40.0  48.36
30993  41.0  48.62
30994  42.0  46.80
30995  43.0  46.38
30996  44.0  43.26
30997  45.0  38.49
30998  46.0  35.36
30999  47.0  34.77
31000  48.0  35.00
31001  49.0  34.72
31002  50.0  3

In [44]:
find_extremums(6, 1999)

The MAX value is: 68.54
The MIN value is: 30.26
...


Ряд VHI за вказаний діапазон років для вказаних областей;

In [45]:
#find in range
def vhi_range(areas, min_year, max_year):
    if not isinstance(areas, list) or not areas:
        return print('No data')
    return df_all[(df_all['Year'] >= min_year) & (df_all['Year'] <= max_year) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [46]:
vhi_range([2, 3], 1998, 2003)

,Year,VHI,area
35201,1998,47.28,2
35202,1998,49.05,2
35203,1998,50.03,2
35204,1998,49.41,2
35205,1998,49.07,2
...,...,...,...
48407,2003,46.43,3
48408,2003,52.15,3
48409,2003,57.91,3
48410,2003,65.84,3


Для всього набору даних виявити роки, протягом яких
екстремальні посухи торкнулися більше вказаного відсотка
областей по Україні (20% областей - 5 областей з 25).

In [47]:
#droughts more than given percent
def extreme_droughts(prc):
    df_drought = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    group = df_drought.groupby('Year')['area'].nunique()
    res = group[group > ((25*prc)/100)].reset_index()
    return res

In [48]:
extreme_droughts(20)

,Year,area
0,2000,6


Аналогічно для помірних посух

In [49]:
def mid_droughts(prc, min=15, max=40):
    df_drought = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    group = df_drought.groupby('Year')['area'].nunique()
    areas = df_all['area'].nunique()
    res = group[group > ((areas* prc)/100)].reset_index()
    return res

In [50]:
mid_droughts(20)

,Year,area
0,1982,27
1,1983,26
2,1984,27
3,1985,27
4,1986,27
5,1987,27
6,1988,27
7,1989,27
8,1990,27
9,1991,26
